# Pageviews for W:PM and W pages with external links
- pageview data source: [`wmf.pageview_hourly`](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Traffic/Pageview_hourly) table
- counts exclude bots/spiders (agent_type = 'user')

In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'
# pandas float formatting
pd.options.display.float_format = '{:.4f}'.format

## Total pageviews for WP:M pages with external links

In [2]:
wpm_query = """
SELECT access_method, SUM(view_count) AS total_pageviews, SUM(view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
WHERE project = 'en.wikipedia'
AND page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY access_method
"""
wpm_pageviews = spark.sql(wpm_query.format(days_in_study,start_date_string, end_date_string))
wpm_pageviews_rdd = wpm_pageviews.rdd
wpm_pageviews_df = sqlContext.createDataFrame(wpm_pageviews_rdd)
wpm_pageviews_pandas = wpm_pageviews_df.toPandas()

wpm_pageviews_pandas

,access_method,total_pageviews,daily_average_pageviews
0,desktop,62650292,1957821.6250
1,mobile web,124214376,3881699.2500
2,mobile app,1150383,35949.4688


## Total pageviews for W pages with external links

In [3]:
w_query = """
SELECT access_method, SUM(view_count) AS total_pageviews, SUM(view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
WHERE project = 'en.wikipedia'
AND page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY access_method
"""
w_pageviews = spark.sql(w_query.format(days_in_study,start_date_string, end_date_string))
w_pageviews_rdd = w_pageviews.rdd
w_pageviews_df = sqlContext.createDataFrame(w_pageviews_rdd)
w_pageviews_pandas = w_pageviews_df.toPandas()

w_pageviews_pandas

,access_method,total_pageviews,daily_average_pageviews
0,desktop,3134600739,97956273.0938
1,mobile web,4120797860,128774933.1250
2,mobile app,54845509,1713922.1562


## Top WP:M pages

In [2]:
# most visited WP:M pages with external links
top_wpm_query = """
SELECT pageview_hourly.page_id, population_page_titles_20190420.page_title, SUM(pageview_hourly.view_count) AS total_pageviews, SUM(pageview_hourly.view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
LEFT JOIN ryanmax.population_page_titles_20190420 ON pageview_hourly.page_id = population_page_titles_20190420.page_id
WHERE project = 'en.wikipedia'
AND pageview_hourly.page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY pageview_hourly.page_id, population_page_titles_20190420.page_title 
ORDER BY total_pageviews DESC
LIMIT 50
"""

top_wpm = spark.sql(top_wpm_query.format(days_in_study,start_date_string, end_date_string))
top_wpm.limit(50).toPandas()

,page_id,page_title,total_pageviews,daily_average_pageviews
0,43573275,Elizabeth Holmes,1349527,42172.7188
1,41779862,Theranos,546969,17092.7812
2,58911,Measles,388340,12135.6250
3,18079,Leonardo da Vinci,380421,11888.1562
4,27546,Sexual intercourse,371290,11602.8125
5,37556,Asperger syndrome,336547,10517.0938
6,791546,Ketogenic diet,319326,9978.9375
7,4488176,Factitious disorder imposed on another,314542,9829.4375
8,4501,Black Death,308057,9626.7812
9,56880920,Ramesh Balwani,305280,9540.0000
